In [2]:
from BeautifulSoup import BeautifulSoup
import urllib2, sys
import re
import itertools
import pandas as pd

In [4]:
#code to request website info
url  = "http://www.menuism.com/restaurant-locations/waffle-house-78955/us/al"
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

req = urllib2.Request(url, headers=hdr)
page = urllib2.urlopen(req)
content = page.read()
soup = BeautifulSoup(content)

In [5]:
#code to grab all the addresses on the page
rawhtml = soup.findAll("ul", {"class":""})

#getting addresses
itemList = []
for item in rawhtml:
    itemList.append(''.join(item.findAll(text=True)).split("\n\n\n"))

#flattening the list and removing white space
chain = itertools.chain(*itemList)
chainedList = (list(chain))
addresses = map(lambda x: x.strip(), chainedList)


#getting cities 
itemList = []
for item in rawhtml:
    itemList.append(item)

#same chaining but this time, i take the raw html tags and flatten them
chain = itertools.chain(*itemList)
chainedList = (list(chain))
y = [item for item in chainedList if len(item) > 1]

#getting final list of cities
cities = []
for i in range(0, len(y)):
    cities.append(y[i].a['title'].split('-')[0].split(" in ")[1])
    
#creating dataframe from lists
data = pd.DataFrame({'raw': addresses,
                     'cities': cities})
data['address'] = map(lambda x: x[x.find(" at ")+4:len(x)], data['raw'])
data['zip'] = map(lambda x: x[16:21], data['raw'])


In [10]:
#Get list of all the cities available
allLinks = soup.find("ul", {"class":"list-unstyled-links"}).findAll("a")

allCities = []
for item in allLinks:
    title = item['title']
    if title not in allCities:
        allCities.append(title)
        
#clean up list of 
newList = map(lambda x: x[16:x.find(" - ")], allCities)

#find cities that does not have multiple locations because we'll have to pull data from them separately
toRemove = map(lambda x:x.strip(), data['cities'].drop_duplicates().tolist())
remainder = [city for city in newList if city not in toRemove]

#mapping the remainder list and pair it with their respective values to get link
newDict = dict(zip(newList, allCities))

finalDict = {}
for key, value in newDict.iteritems():
    if key in remainder:
        finalDict[key] = value
finalDict

#testing
finalDict.values()[0]

u'Waffle House in Eufaula, AL - reviews, directions, menu'

In [7]:
#load up new url from finalDict values 
newUrl = soup.find("a", {"title": finalDict.values()[0]})["href"]

#request the new page
req2 = urllib2.Request(newUrl, headers=hdr)
page2 = urllib2.urlopen(req2)
content2 = page2.read()
soup2 = BeautifulSoup(content2)

#parse address from soup2
addyText = str(soup2.find("span",{"data-listing-attr":"address.street"}).contents)
newAddress = re.sub(r'\\n+\\t\\t\\t\\t\\t\\t\\t\\t', ' ', addyText).split('\\t')[1].replace("']","").strip()

#parsing zipcode from soup2
zipText = str(soup2.find("span",{"data-listing-attr":"address.postal_code"}).contents)
zipCode = re.sub(r'\\n+\\t\\t\\t\\t\\t\\t\\t\\t', ' ', zipText).split('\\t')[1].replace("']","").strip()

#create an array of arrays to append to the dataframe
toAdd = [finalDict.keys()[0], "single loc", newAddress, zipCode]
toAdd

[u'Eufaula, AL', 'single loc', '1551 S Eufaula Ave', '36027']

In [8]:
restaurantList = []
for key, value in finalDict.iteritems():
    #load up new url from finalDict values 
    newUrl = soup.find("a", {"title": value})["href"]

    #request the new page
    req2 = urllib2.Request(newUrl, headers=hdr)
    page2 = urllib2.urlopen(req2)
    content2 = page2.read()
    soup2 = BeautifulSoup(content2)

    #parse address from soup2
    addyText = str(soup2.find("span",{"data-listing-attr":"address.street"}).contents)
    newAddress = re.sub(r'\\n+\\t\\t\\t\\t\\t\\t\\t\\t', ' ', addyText).split('\\t')[1].replace("']","").strip()

    #parsing zipcode from soup2
    zipText = str(soup2.find("span",{"data-listing-attr":"address.postal_code"}).contents)
    zipCode = re.sub(r'\\n+\\t\\t\\t\\t\\t\\t\\t\\t', ' ', zipText).split('\\t')[1].replace("']","").strip()

    #create an array of arrays to append to the dataframe
    toAdd = [key, "single loc", newAddress, zipCode]
    restaurantList.append(toAdd)

restaurantList

[[u'Eufaula, AL', 'single loc', '1551 S Eufaula Ave', '36027'],
 [u'Lincoln, AL', 'single loc', '670 Magnolia St', '35096'],
 [u'Gardendale, AL', 'single loc', '500 Fieldstown Rd', '35071'],
 [u'Evergreen, AL', 'single loc', '1635 Ted Bates Rd', '36401'],
 [u'Fort Payne, AL', 'single loc', '1410 Glenn Blvd Sw', '35968'],
 [u'Daleville, AL', 'single loc', '394 Virginia Ave', '36322'],
 [u'Fultondale, AL', 'single loc', '1334 Walker Chapel Rd', '35068'],
 [u'Trussville, AL', 'single loc', '1128 N Chalkville Rd', '35173'],
 [u'Moody, AL', 'single loc', '2140 Moody Pky', '35004'],
 [u'Greenville, AL', 'single loc', '944 Fort Dale Rd', '36037'],
 [u'Alabaster, AL', 'single loc', '1104 1st Street South', '35007'],
 [u'Sheffield, AL', 'single loc', '4116 N Jackson Hwy', '35660'],
 [u'Madison, AL', 'single loc', '8979 Madison Blvd', '35758'],
 [u'Satsuma, AL', 'single loc', '6097 Highway 43', '36572'],
 [u'Clanton, AL', 'single loc', '2231 7th St S', '35046'],
 [u'Loxley, AL', 'single loc', '1

In [9]:
final = pd.DataFrame(restaurantList)
final.columns = ['cities', 'raw','address','zip']
data.append(final)


,cities,raw,address,zip
0,"Anniston, AL",Waffle House in 36201 at 1201 S Quintard Ave,1201 S Quintard Ave,36201
1,"Anniston, AL",Waffle House in 36201 at 1530 Quintard Ave,1530 Quintard Ave,36201
2,"Attalla, AL",Waffle House in 35954 at 802 Cleveland Ave,802 Cleveland Ave,35954
3,"Attalla, AL",Waffle House in 35954 at 961 Gilbert Ferry Rd Se,961 Gilbert Ferry Rd Se,35954
4,"Auburn, AL",Waffle House in 36830 at 110 W Glenn Ave,110 W Glenn Ave,36830
5,"Auburn, AL",Waffle House in 36830 at 2346 Bent Creek Rd,2346 Bent Creek Rd,36830
6,"Auburn, AL",Waffle House in 36830 at 1738 Opelika Rd,1738 Opelika Rd,36830
7,"Auburn, AL",Waffle House in 36832 at 2167 S College St,2167 S College St,36832
8,"Bessemer, AL",Waffle House in 35022 at 3998 Methodist Cir,3998 Methodist Cir,35022
9,"Bessemer, AL",Waffle House in 35022 at 5010 Academy Ln,5010 Academy Ln,35022


In [12]:
def waffles(state):
#code to request website info
    url  = "http://www.menuism.com/restaurant-locations/waffle-house-78955/us/" + state
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}

    req = urllib2.Request(url, headers=hdr)
    page = urllib2.urlopen(req)
    content = page.read()
    soup = BeautifulSoup(content)
    
    #code to grab all the addresses on the page
    rawhtml = soup.findAll("ul", {"class":""})

    #getting addresses
    itemList = []
    for item in rawhtml:
        itemList.append(''.join(item.findAll(text=True)).split("\n\n\n"))

    #flattening the list and removing white space
    chain = itertools.chain(*itemList)
    chainedList = (list(chain))
    addresses = map(lambda x: x.strip(), chainedList)

    #getting cities 
    itemList = []
    for item in rawhtml:
        itemList.append(item)

    #same chaining but this time, i take the raw html tags and flatten them
    chain = itertools.chain(*itemList)
    chainedList = (list(chain))
    y = [item for item in chainedList if len(item) > 1]

    #getting final list of cities
    cities = []
    for i in range(0, len(y)):
        cities.append(y[i].a['title'].split('-')[0].split(" in ")[1])

    #creating dataframe from lists
    data = pd.DataFrame({'raw': addresses,
                         'cities': cities})
    data['address'] = map(lambda x: x[x.find(" at ")+4:len(x)], data['raw'])
    data['zip'] = map(lambda x: x[16:21], data['raw'])

    #Get list of all the cities available
    allLinks = soup.find("ul", {"class":"list-unstyled-links"}).findAll("a")

    allCities = []
    for item in allLinks:
        title = item['title']
        if title not in allCities:
            allCities.append(title)

    #clean up list of all cities
    newList = map(lambda x: x[16:x.find(" - ")], allCities)

    #find cities that does not have multiple locations because we'll have to pull data from them separately
    toRemove = map(lambda x:x.strip(), data['cities'].drop_duplicates().tolist())
    remainder = [city for city in newList if city not in toRemove]

    #mapping the remainder list and pair it with their respective values to get link
    newDict = dict(zip(newList, allCities))

    finalDict = {}
    for key, value in newDict.iteritems():
        if key in remainder:
            finalDict[key] = value
    
    restaurantList = []
    for key, value in finalDict.iteritems():
        #load up new url from finalDict values 
        newUrl = soup.find("a", {"title": value})["href"]

        #request the new page
        req2 = urllib2.Request(newUrl, headers=hdr)
        page2 = urllib2.urlopen(req2)
        content2 = page2.read()
        soup2 = BeautifulSoup(content2)

        #parse address from soup2
        addyText = str(soup2.find("span",{"data-listing-attr":"address.street"}).contents)
        newAddress = re.sub(r'\\n+\\t\\t\\t\\t\\t\\t\\t\\t', ' ', addyText).split('\\t')[1].replace("']","").strip()

        #parsing zipcode from soup2
        zipText = str(soup2.find("span",{"data-listing-attr":"address.postal_code"}).contents)
        zipCode = re.sub(r'\\n+\\t\\t\\t\\t\\t\\t\\t\\t', ' ', zipText).split('\\t')[1].replace("']","").strip()

        #create an array of arrays to append to the dataframe
        toAdd = [key, "single loc", newAddress, zipCode]
        restaurantList.append(toAdd)
            
    final = pd.DataFrame(restaurantList)
    final.columns = ['cities', 'raw','address','zip']
    data.append(final)


In [13]:
listOfStates = ['al','ak','az','ca','co','de','fl','ga',
               'il','in','ks','ky','la','md','mi','mo',
               'ms','nc','nm','ny','oh','ok','pa','sc',
               'tn','tx','va','wi','wv']